In [1]:
import numpy as np
import time
import scipy
import json
import Additive_mixing_layers_extraction
from scipy.spatial import ConvexHull, Delaunay
import scipy.sparse
import PIL.Image as Image
import RGBXY_method
from numpy import *
Additive_mixing_layers_extraction.DEMO=True

In [2]:
def save_weights(img, palette_rgb, mixing_weights, output_prefix):
    mixing_weights=mixing_weights.reshape((img.shape[0],img.shape[1],-1)).clip(0,1)
    temp=(mixing_weights.reshape((img.shape[0],img.shape[1],-1,1))*palette_rgb.reshape((1,1,-1,3))).sum(axis=2)
    img_diff=temp*255-img*255
    diff=np.square(img_diff.reshape((-1,3))).sum(axis=-1)
    print 'max diff: ', np.sqrt(diff).max()
    print 'median diff', np.median(np.sqrt(diff))
    rmse=np.sqrt(diff.sum()/diff.shape[0])
    print 'RMSE: ', np.sqrt(diff.sum()/diff.shape[0])

    import json
    mixing_weights_filename=output_prefix+"-palette_size-"+str(len(palette_rgb))+"-mixing_weights.js"
    with open(mixing_weights_filename,'wb') as myfile:
        json.dump({'weights': mixing_weights.tolist()}, myfile)

    for i in range(mixing_weights.shape[-1]):
        mixing_weights_map_filename=output_prefix+"-palette_size-"+str(len(palette_rgb))+"-mixing_weights-%02d.png" % i
        Image.fromarray((mixing_weights[:,:,i]*255).round().clip(0,255).astype(np.uint8)).save(mixing_weights_map_filename)
    return rmse

def get_bigger_palette_to_show(palette):
    ##### palette shape is M*3
    c=50
    palette2=np.ones((1*c, len(palette)*c, 3))
    for i in range(len(palette)):
        palette2[:,i*c:i*c+c,:]=palette[i,:].reshape((1,1,-1))
    return palette2


In [3]:
base_dir="./test/"

import glob
filepaths=glob.glob(base_dir+"*.png")
print filepaths



for filepath in filepaths:
    print "#####################"
    print filepath
    img=np.asfarray(Image.open(filepath).convert('RGB'))/255.0
    arr=img.copy()
    X,Y=np.mgrid[0:img.shape[0], 0:img.shape[1]]
    XY=np.dstack((X*1.0/img.shape[0],Y*1.0/img.shape[1]))
    data=np.dstack((img, XY))
    print len(data.reshape((-1,5)))
    
    start=time.time()
    palette_rgb=Additive_mixing_layers_extraction.Hull_Simplification_determined_version(img, filepath[:-4]+"-convexhull_vertices")
    end=time.time()    
    M=len(palette_rgb)
    print "palette size: ", M
    print "palette extraction time: ", end-start
    
    palette_img=get_bigger_palette_to_show(palette_rgb)
    Image.fromarray((palette_img*255).round().astype(np.uint8)).save(filepath[:-4]+"-convexhull_vertices.png")


    ######### for RGBXY RGB black star triangulation.
    start=time.time()
    data_hull=ConvexHull(data.reshape((-1,5)))
    start2=time.time()
    print "convexhull on 5D time: ", start2-start
    mixing_weights_1=Additive_mixing_layers_extraction.Get_ASAP_weights_using_Tan_2016_triangulation_and_then_barycentric_coordinates(img.reshape((-1,3))[data_hull.vertices].reshape((-1,1,3)), palette_rgb, None, order=0)
    mixing_weights_2=Additive_mixing_layers_extraction.recover_ASAP_weights_using_scipy_delaunay(data_hull.points[data_hull.vertices], data_hull.points, option=3)

    mixing_weights=mixing_weights_2.dot(mixing_weights_1.reshape((-1,M)))
    
    end=time.time()
    print "total time: ", end-start
    
    mixing_weights=mixing_weights.reshape((img.shape[0],img.shape[1],-1)).clip(0,1)
    
    output_prefix=filepath[:-4]+'-RGBXY_RGB_black_star_ASAP'
    RMSE=save_weights(arr, palette_rgb, mixing_weights, output_prefix)


['./test/turquoise.png']
#####################
./test/turquoise.png
280800
original hull vertices number:  97
use_quantitized_colors
1483
loop:  0
loop:  10
loop:  20
loop:  30
loop:  40
loop:  50
loop:  60
loop:  70
loop:  80
loop:  90
palette size:  6
palette extraction time:  26.7580060959
convexhull on 5D time:  2.76049113274
[0. 0. 0.]
0.00050687789917 0.000232219696045 5.88893890381e-05
0.000743865966797 0.00019097328186 5.10215759277e-05
0.000753879547119 0.000230073928833 8.70227813721e-05
0.000592231750488 0.000156879425049 3.2901763916e-05
max diff:  7.861846444349073
median diff 4.9227844771419234e-14
RMSE:  0.8387862826258635
5.4658498764 0.678758144379 0.0974788665771
total time:  9.08978700638
max diff:  7.861846444349664
median diff 6.394884621840902e-14
RMSE:  0.5309361214629531
